# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6708321f70>
├── 'a' --> tensor([[ 2.0359,  0.0232,  0.6186],
│                   [ 1.7094, -0.1522,  0.2298]])
└── 'x' --> <FastTreeValue 0x7f6708321f40>
    └── 'c' --> tensor([[-0.0558,  0.3715,  0.0592, -0.6423],
                        [-1.7712,  0.4956,  0.3124,  0.7022],
                        [-0.3167,  1.1362, -1.2945,  1.1934]])

In [4]:
t.a

tensor([[ 2.0359,  0.0232,  0.6186],
        [ 1.7094, -0.1522,  0.2298]])

In [5]:
%timeit t.a

62.5 ns ± 0.103 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6708321f70>
├── 'a' --> tensor([[-0.3736, -0.0759, -0.0618],
│                   [-0.0675,  0.7589,  1.4566]])
└── 'x' --> <FastTreeValue 0x7f6708321f40>
    └── 'c' --> tensor([[-0.0558,  0.3715,  0.0592, -0.6423],
                        [-1.7712,  0.4956,  0.3124,  0.7022],
                        [-0.3167,  1.1362, -1.2945,  1.1934]])

In [7]:
%timeit t.a = new_value

66.7 ns ± 0.0609 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.0359,  0.0232,  0.6186],
               [ 1.7094, -0.1522,  0.2298]]),
    x: Batch(
           c: tensor([[-0.0558,  0.3715,  0.0592, -0.6423],
                      [-1.7712,  0.4956,  0.3124,  0.7022],
                      [-0.3167,  1.1362, -1.2945,  1.1934]]),
       ),
)

In [10]:
b.a

tensor([[ 2.0359,  0.0232,  0.6186],
        [ 1.7094, -0.1522,  0.2298]])

In [11]:
%timeit b.a

58.6 ns ± 0.124 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.7691, -0.5225, -0.8413],
               [ 0.9592,  1.0582,  0.4506]]),
    x: Batch(
           c: tensor([[-0.0558,  0.3715,  0.0592, -0.6423],
                      [-1.7712,  0.4956,  0.3124,  0.7022],
                      [-0.3167,  1.1362, -1.2945,  1.1934]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.0877 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

814 ns ± 0.165 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 19.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 469 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f66505abf40>
├── 'a' --> tensor([[[ 2.0359,  0.0232,  0.6186],
│                    [ 1.7094, -0.1522,  0.2298]],
│           
│                   [[ 2.0359,  0.0232,  0.6186],
│                    [ 1.7094, -0.1522,  0.2298]],
│           
│                   [[ 2.0359,  0.0232,  0.6186],
│                    [ 1.7094, -0.1522,  0.2298]],
│           
│                   [[ 2.0359,  0.0232,  0.6186],
│                    [ 1.7094, -0.1522,  0.2298]],
│           
│                   [[ 2.0359,  0.0232,  0.6186],
│                    [ 1.7094, -0.1522,  0.2298]],
│           
│                   [[ 2.0359,  0.0232,  0.6186],
│                    [ 1.7094, -0.1522,  0.2298]],
│           
│                   [[ 2.0359,  0.0232,  0.6186],
│                    [ 1.7094, -0.1522,  0.2298]],
│           
│                   [[ 2.0359,  0.0232,  0.6186],
│                    [ 1.7094, -0.1522,  0.2298]]])
└── 'x' --> <FastTreeValue 0x7f66505d3f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 48.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f670831abe0>
├── 'a' --> tensor([[ 2.0359,  0.0232,  0.6186],
│                   [ 1.7094, -0.1522,  0.2298],
│                   [ 2.0359,  0.0232,  0.6186],
│                   [ 1.7094, -0.1522,  0.2298],
│                   [ 2.0359,  0.0232,  0.6186],
│                   [ 1.7094, -0.1522,  0.2298],
│                   [ 2.0359,  0.0232,  0.6186],
│                   [ 1.7094, -0.1522,  0.2298],
│                   [ 2.0359,  0.0232,  0.6186],
│                   [ 1.7094, -0.1522,  0.2298],
│                   [ 2.0359,  0.0232,  0.6186],
│                   [ 1.7094, -0.1522,  0.2298],
│                   [ 2.0359,  0.0232,  0.6186],
│                   [ 1.7094, -0.1522,  0.2298],
│                   [ 2.0359,  0.0232,  0.6186],
│                   [ 1.7094, -0.1522,  0.2298]])
└── 'x' --> <FastTreeValue 0x7f6650cc60d0>
    └── 'c' --> tensor([[-0.0558,  0.3715,  0.0592, -0.6423],
                        [-1.7712,  0.4956,  0.3124,  0.7022],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 31.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.8 µs ± 207 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.0558,  0.3715,  0.0592, -0.6423],
                       [-1.7712,  0.4956,  0.3124,  0.7022],
                       [-0.3167,  1.1362, -1.2945,  1.1934]],
              
                      [[-0.0558,  0.3715,  0.0592, -0.6423],
                       [-1.7712,  0.4956,  0.3124,  0.7022],
                       [-0.3167,  1.1362, -1.2945,  1.1934]],
              
                      [[-0.0558,  0.3715,  0.0592, -0.6423],
                       [-1.7712,  0.4956,  0.3124,  0.7022],
                       [-0.3167,  1.1362, -1.2945,  1.1934]],
              
                      [[-0.0558,  0.3715,  0.0592, -0.6423],
                       [-1.7712,  0.4956,  0.3124,  0.7022],
                       [-0.3167,  1.1362, -1.2945,  1.1934]],
              
                      [[-0.0558,  0.3715,  0.0592, -0.6423],
                       [-1.7712,  0.4956,  0.3124,  0.7022],
                       [-0.3167,  1.1362, -1.2945,  1.1934]],

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 366 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.0558,  0.3715,  0.0592, -0.6423],
                      [-1.7712,  0.4956,  0.3124,  0.7022],
                      [-0.3167,  1.1362, -1.2945,  1.1934],
                      [-0.0558,  0.3715,  0.0592, -0.6423],
                      [-1.7712,  0.4956,  0.3124,  0.7022],
                      [-0.3167,  1.1362, -1.2945,  1.1934],
                      [-0.0558,  0.3715,  0.0592, -0.6423],
                      [-1.7712,  0.4956,  0.3124,  0.7022],
                      [-0.3167,  1.1362, -1.2945,  1.1934],
                      [-0.0558,  0.3715,  0.0592, -0.6423],
                      [-1.7712,  0.4956,  0.3124,  0.7022],
                      [-0.3167,  1.1362, -1.2945,  1.1934],
                      [-0.0558,  0.3715,  0.0592, -0.6423],
                      [-1.7712,  0.4956,  0.3124,  0.7022],
                      [-0.3167,  1.1362, -1.2945,  1.1934],
                      [-0.0558,  0.3715,  0.0592, -0.6423],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 418 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

324 µs ± 2.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
